In [1]:
from sklearn.model_selection import train_test_split  
from sklearn import preprocessing  
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model 
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_absolute_error,mean_squared_error 

Using TensorFlow backend.


In [2]:
def create_model(feature):
    model_name = 'LSTM'
    model="model LSTM"
    model = Sequential()
    model.add(LSTM(return_sequences=True, input_shape=(None, feature), units=50))
    model.add(Dropout(0.2))
    model.add(LSTM(100, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    model.add(Activation('linear')) 
    model.compile(loss='mae', optimizer='adam')
    return model,model_name

In [3]:
def set_features(data):
    data_csv = data[['date','high','low','open','close','volume','quoteVolume','weightedAverage','Average','dates_formatted']]
    data_csv=data[['open','high','low','close','volume']] 
    df = data_csv 
    
    df_data_shift=df.shift(-1)
    df_data_label =  df[['close']] 

    # adjusting the shape of both
    df.drop(df.index[len(df)-1], axis=0, inplace=True)
    df_data_label.drop(df_data_label.index[len(df_data_label)-1], axis=0, inplace=True) 

    # conversion to numpy array
    x, y = df.values, df_data_label.values  
    
    # scaling values for model
    x_scale = MinMaxScaler()
    y_scale = MinMaxScaler() 
    #print(x_scale)
    X = x_scale.fit_transform(x)
    Y = y_scale.fit_transform(y.reshape(-1,1))   
    return X,Y,df,y_scale,x_scale

In [4]:
my_file = 'BitCoin_Data.csv'
data = pd.read_csv(my_file,index_col=0) 
data.head()

,date,high,low,open,close,volume,quoteVolume,weightedAverage,Average,dates_formatted
44352,1504204200,4737.267364,4710.500000,4718.000000,4737.267364,680082.7339,143.942568,4724.681116,4724,2017-09-01
44353,1504206000,4742.418945,4700.000000,4737.267364,4716.039868,653827.5575,138.460804,4722.112961,4722,2017-09-01
44354,1504207800,4722.990000,4700.000000,4716.039868,4710.096467,501252.4665,106.347592,4713.341020,4713,2017-09-01
44355,1504209600,4720.000000,4701.000000,4710.096467,4719.826383,310109.5252,65.849511,4709.367158,4709,2017-09-01
44356,1504211400,4736.000000,4710.166319,4719.826383,4726.000000,322813.4876,68.314698,4725.388482,4725,2017-09-01


In [5]:
X,Y,df,y_scale,x_scale =set_features(data) 

/home/aclab/.local/lib/python3.5/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


# Train Test split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)

# Data Reshape

In [7]:
X_train = X_train.reshape((-1,1,df.shape[1]))
X_test = X_test.reshape((-1,1,df.shape[1])) 

# Train Model

In [8]:
model,model_name=create_model(df.shape[1])

In [9]:
hist=model.fit(X_train,y_train,batch_size=250, epochs=10, validation_split=0.05, verbose=1) 

Train on 2992 samples, validate on 158 samples
Epoch 1/10
2992/2992 [==============================] - 1s 463us/step - loss: 0.4061 - val_loss: 0.3409
Epoch 2/10
2992/2992 [==============================] - 0s 18us/step - loss: 0.2897 - val_loss: 0.1978
Epoch 3/10
2992/2992 [==============================] - 0s 17us/step - loss: 0.1407 - val_loss: 0.0806
Epoch 4/10
2992/2992 [==============================] - 0s 17us/step - loss: 0.0982 - val_loss: 0.0701
Epoch 5/10
2992/2992 [==============================] - 0s 18us/step - loss: 0.0775 - val_loss: 0.0559
Epoch 6/10
2992/2992 [==============================] - 0s 17us/step - loss: 0.0577 - val_loss: 0.0345
Epoch 7/10
2992/2992 [==============================] - 0s 17us/step - loss: 0.0406 - val_loss: 0.0140
Epoch 8/10
2992/2992 [==============================] - 0s 17us/step - loss: 0.0347 - val_loss: 0.0156
Epoch 9/10
2992/2992 [==============================] - 0s 17us/step - loss: 0.0336 - val_loss: 0.0160
Epoch 10/10
2992/2992 [==

# Model Prediction

In [10]:
pred_pred=model.predict(X_test)  
print("MSE:", mean_squared_error(pred_pred,y_test))

MSE: 0.00019513256638862606
